In [ ]:
# !!! MODIFY ME !!! 
# The total number of nodes can be at most 10. To use more, one must extend the 'names' list below
N_VALIDATORS = 7
N_ARCHIVISTS = 2
# True = use all nodes as bootnodes
# False = use only archivists as bootnodes
ALL_BOOTNODES = False

# Path to aleph-node repo
ALEPH_NODE = '$HOME/aleph/aleph-node/'
# Path to test directory
TEST_HOME = '$HOME/aleph/tests'

from os.path import join, expandvars
# Path to working directory, where chainspec, logs and nodes' dbs are written:
# WARNING: the whole workdir is cleared before the chain is set up!
workdir = expandvars(join(TEST_HOME, 'workdir'))
# Path to the pre-update aleph-node binary:
oldbin = expandvars(join(TEST_HOME, 'aleph-node-10.1'))
# Path to the post-update aleph-node binary:
newbin = expandvars(join(TEST_HOME, 'aleph-node-11.3'))
# Path to the post-update compiled runtime:
runtime = expandvars(join(TEST_HOME, 'aleph_runtime.64'))

In [ ]:
import shutil
import sys
import time
from substrateinterface import SubstrateInterface, Keypair

sys.path.append(join(ALEPH_NODE, 'local-tests'))
from chainrunner import Chain, Seq, generate_keys

In [ ]:
# Ports for node 0, consecutive numbers are used for other nodes 
PORT = 30334
WS_PORT = 9943
RPC_PORT = 9933
VAL_PORT = 30343

In [ ]:
names = ['//Alice','//Bob','//Charlie','//Dave','//Eve','//Ferdie','//George','//Hans','//Iris','//James']
val_keys = generate_keys(oldbin, names[:N_VALIDATORS])
arch_keys = generate_keys(oldbin, names[N_VALIDATORS:N_VALIDATORS + N_ARCHIVISTS])

sudo = val_keys['//Alice']
val_addrs = ['127.0.0.1:'+str(i) for i in range(VAL_PORT, VAL_PORT + N_VALIDATORS + N_ARCHIVISTS)]

shutil.rmtree(workdir, True)
chain = Chain(workdir)
chain.bootstrap(oldbin,
                val_keys.values(),
                nonvalidators=arch_keys.values(),
                chain_type='live',
                sudo_account_id=sudo,
                raw=True)

chain.set_flags('validator',
                'unsafe-ws-external',
                'unsafe-rpc-external',
                'no-mdns',
                port=Seq(PORT),
                rpc_port=Seq(RPC_PORT),
                validator_port=Seq(VAL_PORT),
                public_validator_addresses=val_addrs,
                unit_creation_delay=500,
                execution='Native',
                rpc_cors='all',
                rpc_methods='Unsafe',
                state_pruning='archive',
               )

addresses = [n.address() for n in chain]
bootnodes = addresses if ALL_BOOTNODES else addresses[N_VALIDATORS:]
chain.set_flags(bootnodes=' '.join(bootnodes), public_addr=addresses)

In [ ]:
# Helper function to upgrade nodes
def upgrade_node(i, purge=False, wait=True):
    prev = chain[i].change_binary(newbin, 'new', purge)
    time.sleep(5)
    if wait:
        try:
            chain.wait_for_finalization(prev, nodes=[i], timeout=120, finalized_delta=7)
            print(f'Node {i} finalization restored')
        except TimeoutError:
            print(f'Node {i} finalization stuck')
    chain.status()

In [ ]:
chain.start('old')
chain.wait_for_finalization(0)
chain.status()

In [ ]:
upgrade_node(0, wait=False)

In [ ]:
upgrade_node(1)

In [ ]:
upgrade_node(2, purge=True)

In [ ]:
upgrade_node(3)

In [ ]:
upgrade_node(4)

In [ ]:
upgrade_node(5)

In [ ]:
upgrade_node(6)

In [ ]:
upgrade_node(7)

In [ ]:
upgrade_node(8)

In [ ]:
chain[1].stop()
chain[1].start('postcrash_')

In [ ]:
chain[2].update_runtime(runtime, names[0])

In [ ]:
chain.stop()

In [ ]:
# TOOLBOX

In [ ]:
# General chain status
chain.status()

In [ ]:
# Restart some nodes
chain.stop(nodes=[2,3])
time.sleep(5)
chain.start('restart', nodes=[2,3])

In [ ]:
# Restart the whole chain
chain.stop()
time.sleep(5)
chain.start('fullrestart')

In [ ]:
# Change log level for one node (on the fly)
chain[6].set_log_level('sync', 'debug')

In [ ]:
# Change log level for all nodes (on the fly)
chain.set_log_level('sync', 'debug')

In [ ]:
# Grep node's current log for regexp
chain[0].greplog(r'best: #\d+ .+ finalized #\d+')